In [242]:
import django_initializer
from datetime import datetime
import arrow
from api.utils.common.verify import Verify
from api.utils.error_handle.error.api_error import ApiVerifyError
from backend.pymongo.mongodb import db
import openpyxl
import xlsxwriter, os.path 
from api.models.campaign.campaign import Campaign
from api.models.campaign.campaign_comment import CampaignComment
from api.models.campaign.campaign_product import CampaignProduct
from api.models.order.order import Order
from api.models.order.pre_order import PreOrder
from backend.pymongo.mongodb import db

import pandas as pd
from django.db.models import Count, Q, Sum, ExpressionWrapper, IntegerField, F, Subquery, PositiveIntegerField
class SubqueryCount(Subquery):
    template = "(SELECT count(*) FROM (%(subquery)s) _count)"
    output_field = PositiveIntegerField()


class SubquerySum(Subquery):
    template = '(SELECT sum(_sum."%(column)s") FROM (%(subquery)s) _sum)'

    def __init__(self, queryset, column, output_field=None, **extra):
        if output_field is None:
            output_field = queryset.model._meta.get_field(column)
        super().__init__(queryset, output_field, column=column, **extra)

In [294]:
class SalesReport:
    def __init__(self):
        pass
    
    @classmethod
    def change_time_format(cls, time):
        return time.format('YYYY-MM-DD HH:mm:ss')
    @classmethod
    def Q2D(cls, queryset):
        return pd.DataFrame.from_records(queryset.values())
    
    @classmethod
    def get_period_campaign_queryset(cls, start_time, end_time):
        beginning_of_day_of_start_time = cls.change_time_format(arrow.get(start_time).replace(hour=0, minute=0, second=0))
        end_of_day_of_end_time = cls.change_time_format(arrow.get(end_time).replace(hour=23, minute=59, second=59))
        
        campaign_qs = Campaign.objects.filter(start_at__gte=beginning_of_day_of_start_time, end_at__lte=end_of_day_of_end_time)
        campaign_qs = campaign_qs.annotate(
            post_comment = Count("comments"),
            
        )
        campaign_product_qs = CampaignProduct.objects.filter(Q(campaign__start_at__gte=beginning_of_day_of_start_time)&Q(campaign__end_at__lte=end_of_day_of_end_time))
        campaign_comment_qs = CampaignComment.objects.filter(Q(campaign__start_at__gte=beginning_of_day_of_start_time)&Q(campaign__end_at__lte=end_of_day_of_end_time))
        pre_order_qs = PreOrder.objects.filter(Q(campaign__start_at__gte=beginning_of_day_of_start_time)&Q(campaign__end_at__lte=end_of_day_of_end_time))
        order_qs = Order.objects.filter(Q(campaign__start_at__gte=beginning_of_day_of_start_time)&Q(campaign__end_at__lte=end_of_day_of_end_time))
        
        campaign_df = cls.Q2D(campaign_qs)
        campaign_product_df = cls.Q2D(campaign_product_qs)
        campaign_comment_df = cls.Q2D(campaign_comment_qs)
        pre_order_df = cls.Q2D(pre_order_qs)
        order_df = cls.Q2D(order_qs)
        
        return campaign_df, campaign_product_df, campaign_comment_df, pre_order_df, order_df
    
    def get_basic_report():
        pass

In [333]:
start_time = "2022-06-15 00:00:00"
end_time = "2022-06-15"
beginning_of_day_of_start_time = arrow.get(start_time).replace(hour=0, minute=0, second=0).datetime
end_of_day_of_end_time = arrow.get(end_time).replace(hour=23, minute=59, second=59).datetime
display(beginning_of_day_of_start_time)
display(end_of_day_of_end_time)
cursor=db.api_campaign.aggregate([
    {
        "$match":{
            "start_at":{
                "$gte":beginning_of_day_of_start_time
            },
            "end_at":{
                "$lte":end_of_day_of_end_time
            }
        }
    },
    {
        "$lookup": {
            "from": "api_campaign_comment",
            "as": "campaign_comment",
            'let': {'id': "$id" },
            "pipeline":[
                {
                    "$match":{
                        '$expr': { '$eq': ["$$id", "$campaign_id"] },
                        "id":{"$ne":None}
                    }
                }
            ]
        },
    },
    {
        "$lookup": {
            "from": "api_campaign_product",
            "as": "campaign_product_group",
            'let': {'id': "$id" },
            "pipeline":[
                {
                    "$match":{
                        '$expr': { '$eq': ["$$id", "$campaign_id"] },
                        "id":{"$ne":None}
                    }
                },
                {
                    "$group": { '_id': '$campaign_id', 'unique_order_code': { "$addToSet": "$order_code"}}
                },
                {
                    "$project":{
                        "_id":0,
                        "campaign_id":1,
                        "unique_order_code":1
                    }
                }
                
            ]
        },
    },
    {
        "$lookup": {
            "from": "api_campaign_comment",
            "as": "campaign_comment_3",
            'let': {'id': "$id" },
            "pipeline":[
                {
                    "$match":{
                        '$expr': { '$in': ["message","$campaign_product_group.unique_order_code"] },
                        "id":{"$ne":None}
                    }
                }
            ]
        },
    },
    {
        "$lookup": {
            "from": "api_campaign_product",
            "as": "campaign_product",
            'let': {'id': "$id" },
            "pipeline":[
                {
                    "$match":{
                        '$expr': { '$eq': ["$$id", "$campaign_id"] },
                        "id":{"$ne":None}
                    }
                }
                
            ]
        },
    },
    {
        "$project":{
            "_id":0,
            "id": 1,
            "post_comments": {"$size":"$campaign_comment.campaign_id"},
            "total_no_of_items": {"$size":"$campaign_product.campaign_id"},
        }
    }
])
l = list(cursor)

datetime.datetime(2022, 6, 15, 0, 0, tzinfo=tzutc())

datetime.datetime(2022, 6, 15, 23, 59, 59, tzinfo=tzutc())

In [334]:
l

[{'id': 495, 'post_comments': 2, 'total_no_of_items': 3},
 {'id': 496, 'post_comments': 4, 'total_no_of_items': 6},
 {'id': 497, 'post_comments': 14, 'total_no_of_items': 2},
 {'id': 499, 'post_comments': 13, 'total_no_of_items': 4},
 {'id': 501, 'post_comments': 16, 'total_no_of_items': 3},
 {'id': 502, 'post_comments': 4, 'total_no_of_items': 4}]